## Создаем соединение с базой данных с помощью библиотеки sqlite3

In [1]:
import sqlite3
import pandas as pd

sql_connection=sqlite3.connect('../../data/checking-logs.sqlite')
sql_connection

## Используя только один запрос для каждой из групп, создаем два датафрейма: test_results и control_results со столбцами time и avg_diff и только двумя строками.


1. times имеет значения before и after.

2. avg_diff содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
3. Учитываются только те пользователи, для которых имеются наблюдения и before, и after.


4. По-прежнему не используем лабораторное задание 'project1'

In [2]:
test_results  = '''
        SELECT time, AVG(avg_diff)
        FROM (SELECT uid, labname, AVG ((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff,
            CASE
                WHEN (strftime('%s', first_commit_ts)) < (strftime('%s', first_view_ts))
                THEN 'before' ELSE ' after' END AS time
            FROM test
            LEFT JOIN deadlines ON labname = labs WHERE labname NOT LIKE 'project1'
            GROUP BY uid, time)
        WHERE uid IN (SELECT uid
                    FROM (SELECT uid, 
                        CASE
                            WHEN (strftime('%s', first_commit_ts)) < (strftime('%s', first_view_ts))
                            THEN 'before' ELSE ' after' END AS time
                    FROM test
                    LEFT JOIN deadlines ON labname = labs WHERE labname NOT LIKE 'project1'
                    GROUP BY uid, time)
        GROUP BY uid
        HAVING COUNT(uid) == 2)
        GROUP BY time'''

test_results = pd.read_sql(test_results, sql_connection)
test_results

,time,AVG(avg_diff)
0,after,-99.523810
1,before,-66.047619


In [3]:
control_results = '''
        SELECT time, AVG(avg_diff)
        FROM (SELECT uid, labname, AVG ((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff,
            CASE
                WHEN (strftime('%s', first_commit_ts)) < (strftime('%s', first_view_ts))
                THEN 'before' ELSE ' after' END AS time
            FROM control
            LEFT JOIN deadlines ON labname = labs WHERE labname NOT LIKE 'project1'
            GROUP BY uid, time)
        WHERE uid IN (SELECT uid
                    FROM (SELECT uid, 
                        CASE
                            WHEN (strftime('%s', first_commit_ts)) < (strftime('%s', first_view_ts))
                            THEN 'before' ELSE ' after' END AS time
                    FROM control
                    LEFT JOIN deadlines ON labname = labs WHERE labname NOT LIKE 'project1'
                    GROUP BY uid, time)
        GROUP BY uid
        HAVING COUNT(uid) == 2)
        GROUP BY time'''

control_results = pd.read_sql(control_results, sql_connection)

control_results

,time,AVG(avg_diff)
0,after,-99.322222
1,before,-98.033333


## Закрываем соединение с базой данных.

In [4]:
sql_connection.close()